<a href="https://colab.research.google.com/github/Daisy-yww/github/blob/main/Spaceship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv("/content/drive/MyDrive/spaceship-titanic/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/spaceship-titanic/test.csv")
test_ids = test_data["PassengerId"]

In [2]:
train_data.head(10)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [3]:
test_data.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
5,0027_01,Earth,False,F/7/P,TRAPPIST-1e,31.0,False,0.0,1615.0,263.0,113.0,60.0,Karlen Ricks
6,0029_01,Europa,True,B/2/P,55 Cancri e,21.0,False,0.0,NaN,0.0,0.0,0.0,Aldah Ainserfle
7,0032_01,Europa,True,D/0/S,TRAPPIST-1e,20.0,False,0.0,0.0,0.0,0.0,0.0,Acrabi Pringry
8,0032_02,Europa,True,D/0/S,55 Cancri e,23.0,False,0.0,0.0,0.0,0.0,0.0,Dhena Pringry
9,0033_01,Earth,False,F/7/S,55 Cancri e,24.0,False,0.0,639.0,0.0,0.0,0.0,Eliana Delazarson


In [4]:
def clean(data):
    data =  data.drop(["PassengerId", "HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name"], axis=1)

    cols = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Age"]
    for col in cols:
          data[col].fillna(data[col].mean(), inplace=True)
    return data

In [5]:
train_data = clean(train_data)
test_data = clean(test_data) 
X = train_data.iloc[:, :-1]
Y = train_data.iloc[:, -1]
print(f"train data shape: {X.shape}")
print(f"test data shape: {test_data.shape}")

train data shape: (8693, 6)
test data shape: (4277, 6)


In [6]:
'''
concat_data = pd.concat([X, test_data], axis=0)
dummy_data = pd.get_dummies(concat_data)    
dummy_train_data = dummy_data.iloc[:X.shape[0], :]
dummy_test_data = dummy_data.iloc[X.shape[0]:, :]
'''

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
sc_x = scaler.fit_transform(X)
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

In [10]:
depth = []

for i in tqdm(range(2,15,1)):
    tree = RandomForestClassifier(criterion="entropy", max_depth = i, random_state=10)
    scores = cross_val_score(tree, sc_x, Y, cv=5)
    depth.append(scores.mean())
depth = np.array(depth)
final_depth = np.argmax(depth) + 2

100%|██████████| 13/13 [00:50<00:00,  3.89s/it]


In [11]:
print(final_depth)

7


In [12]:
final_tree = RandomForestClassifier(criterion="entropy", max_depth = final_depth, random_state=10)
final_tree.fit(sc_x, Y)

RandomForestClassifier(criterion='entropy', max_depth=7, random_state=10)

In [13]:
sc_test_data_x = scaler.transform(test_data)

In [14]:
submission = final_tree.predict(sc_test_data_x)
print(submission)

[ True False  True ...  True  True  True]


In [15]:
submission_data = pd.DataFrame({"PassengerId": test_ids.values, "Transported": submission })
print(submission_data)

     PassengerId  Transported
0        0013_01         True
1        0018_01        False
2        0019_01         True
3        0021_01         True
4        0023_01         True
...          ...          ...
4272     9266_02         True
4273     9269_01        False
4274     9271_01         True
4275     9273_01         True
4276     9277_01         True

[4277 rows x 2 columns]


In [16]:
submission_data.to_csv("/content/drive/MyDrive/spaceship-titanic/submission.csv", index=False)